# This notebook is for verifying the genrated internal ONNX graphs generated during the flow before the HW Conversion
### The main base for the verification process is the FINN function  finn.core.onnx_exec.execute_onnx

## First Step:
Importing important libraries and base files

In [1]:
from finn.util.basic import make_build_dir
from finn.util.visualization import showSrc, showInNetron
import os

build_dir = '/home/drmervat/Desktop/YOLO_GP/YOLO__VER'
onnx_dir = '/home/drmervat/Desktop/YOLO_GP/YOLO__'

In [2]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 1.3 MB/s eta 0:00:0000:0100:02m


## Validating the output
In this cell you can check the output values from each generated ONNX graph and compare it with the original model. Just replace the onnx file in line 35 with yours

In [14]:
#using detect class
import glob
import cv2
import numpy as np
test_img_paths = glob.glob("stimulusgrid.jpg")

for number, test_img_path in enumerate(test_img_paths):
    img_org = cv2.imread(test_img_path)
    img = img_org.copy()
    img
    # Define the desired dimensions for the resized image
    width = 416
    height = 416

    # # Resize the image
    resized_image = cv2.resize(img, (width, height))

    # Save the resized image
    cv2.imwrite('stimulusgrid_resized.jpg', resized_image)
    resized_image = resized_image[:, :, ::-1]
    resized_image = resized_image.astype(np.float32)
    resized_image_ip = resized_image.astype(np.uint8)
    # resized_image_ip = resized_image_ip.transpose(2,1,0)
    driver_in = np.expand_dims(resized_image_ip, 0)
    driver_in = driver_in.astype(np.float32)
    driver_in = driver_in.transpose(0,3,1,2)
    # driver_in_ip = np.expand_dims(resized_image_ip, 0)
    # driver_in_ip = driver_in_ip.transpose(0,3,1,2)
    #ready the input
    import numpy as np
    import onnx.numpy_helper as nph
    from qonnx.core.modelwrapper import ModelWrapper
    input_dict = {"global_in": (driver_in)}
    #simulating
    model_for_sim = ModelWrapper(onnx_dir+"/yolo_streamlined.onnx")
    import finn.core.onnx_exec as oxe
    output_dict = oxe.execute_onnx(model_for_sim, input_dict, return_full_exec_context=False)
    output_pysim = output_dict[list(output_dict.keys())[0]]
    output_pysim=output_pysim.transpose(0,2,3,1)
    output_pysim_arr=np.array(output_pysim)
    conf = output_pysim_arr[0,:,:,10]/2+0.5
    conf=conf * (output_pysim_arr[0,:,:,11]/2+0.5)
    print(conf.max())
    # nc  = 1
    # anchors = np.array([[10,14,23,27,37,58]]) / np.array([32])
    # torch.Tensor(output_pysim)
    # detect_head = Detect(nc, anchors)
    # pred = detect_head([output_pysim])[0]
    # pred = non_max_suppression(pred, conf_thres=0.3, iou_thres=0.10, classes=None, max_det=10)
    # print(np.array(pred).shape)
    # #print(pred[0,0:12,4])
    # #arr = (np.array(pred[0,:,4]))
    # arr = (np.array(pred))
    # max_val = arr[0,:,4].max()  # or np.max(arr)
    # min_val = arr[0,:,4].min()  # or np.min(arr)
    # print(max_val, min_val)
    

0.47842395
